# Assignment 6

1. Use yeast dataset from UCI http://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data

In [1]:
pip install numpy --upgrade

Requirement already up-to-date: numpy in c:\users\melis\anaconda3\lib\site-packages (1.21.4)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Loading the dataset
import requests
from io import BytesIO
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data'
s = requests.get(url).content
df = pd.read_fwf(BytesIO(s), header = None)

In [4]:
# Showing the first 5 rows
df.head()

,0,1,2,3,4,5,6,7,8,9
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


2. Remove the first column and use the last column as the target. 3. Only leave CYT and VAC classes

In [5]:
df = df[(df.iloc[:,-1] == 'VAC') | (df.iloc[:,-1] == 'CYT')]
target = df.iloc[:, -1]
df = df.iloc[: , 1:9]

4. Replace [0.3, 0.5, 0.7] in feature 2 to null

In [6]:
df.iloc[:,1].replace([0.3, 0.5, 0.7], np.nan, inplace = True)

5. Replace [0.26, 0.36, 0.64] in feature 3 to null

In [7]:
df.iloc[:,2].replace([0.26, 0.36, 0.64], np.nan, inplace = True)

6. Split the data

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, target, random_state=42)

7. Impute the data

In [9]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(X_train)
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

8. Build a outlier detection model to classify VAC from CYT, i.e. 0 from 1

In [10]:
from sklearn.ensemble import IsolationForest
from numpy import quantile, where, random
iforest = IsolationForest(contamination=.01) # contamination refers to the portion of outliers in the data
pred = iforest.fit_predict(X_train)

In [11]:
mask = pred != -1
X_train, y_train = X_train.loc[mask, :], y_train.loc[mask]

9. Build a classifer using sample augmentation techniques to flassify VAC from CYT, i.e. 0 from 1

In [12]:
!pip install imblearn --user

In [13]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek(sampling_strategy='auto')
X_smt, y_smt = smt.fit_resample(X_train, y_train)

10. Try different methods and hyper paramters

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(SVC(), param_grid=param_grid, cv=5)
grid.fit(X_smt, y_smt)

print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Best set score: {:.2f}".format(grid.score(X_smt, y_smt)))
print("Best parameters: ", grid.best_params_)

Best cross-validation accuracy: 0.94
Best set score: 0.99
Best parameters:  {'C': 10, 'gamma': 100}


In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

model = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]

grid = dict(learning_rate=learning_rate, n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_search.fit(X_smt, y_smt)

print("Best cross-validation accuracy: {:.2f}".format(grid_search.best_score_))
print("Best set score: {:.2f}".format(grid_search.score(X_smt, y_smt)))
print("Best parameters: ", grid_search.best_params_)

Best cross-validation accuracy: 0.97
Best set score: 1.00
Best parameters:  {'learning_rate': 0.1, 'n_estimators': 1000}


Gradient Boosting Classifier has a higher cross-validation accuracy.

In [16]:
model = GradientBoostingClassifier(learning_rate = 0.1, n_estimators = 1000)
model.fit(X_smt, y_smt)
y_pred = model.predict(X_test)

11. Report perfromance using F-1 score

In [17]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='weighted')

0.8874612720976854